# Homesite

In [1]:
import pandas as pd
import numpy as np
import copy
import csv
from sklearn import linear_model
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures


Using Theano backend.


In [2]:
def ReplaceWithDummies(column, *DataFrames):
    #The purpose of this function is to replace a column of type 'object' with n distict values,
    #common to all DataFrames passed in
    #For example train and test data sets, with n-1 boolean columns as delete the original culmnn
    for df in DataFrames: #Make sure the column is actually in all data frames
        if column not in df.columns:
            print('column not found')
            return None
    size=[]
    for df in DataFrames:
        size.append(df.shape[0])
    
    long_column=[]
    for i in range(len(DataFrames)):
        long_column.append(DataFrames[i][column])
    long_column = pd.concat(long_column)
    dummies = pd.get_dummies(long_column)
    dummies.drop(list(dummies.columns)[0], axis=1, inplace=True) # dropping one column from dummies
    
    Dummies =[] # As list of dummies to append to the list of DataFrames in order 
    for s in size:
        Dummies.append(dummies[:s])
        dummies=dummies[s:]
    
    #drop the column that needs replacing
    for df in DataFrames:
        df.drop(column, axis=1, inplace=True)

    
    #Now append the dummy variables:

    for i,df in enumerate(DataFrames):
        for column_type in Dummies[i]:     
            new_name=str(column) +'_'+ str(column_type)
            df[new_name]=Dummies[i][column_type]
    return DataFrames
    
                

In [23]:
tr=pd.read_csv('train.csv')
te=pd.read_csv('test.csv')

In [24]:
#Run for local testing
n= len(tr)
n = int(n*(float(2)/float(3)))
train = copy.deepcopy(tr[:n])
m=int((len(tr)-n)/2)
validation = copy.deepcopy(tr[n:n+m])
test = copy.deepcopy(tr[n+m:])



In [36]:
#Use this for the real thing
train = tr[:]
validation = te[:]
test=te[:]

In [37]:
#Converts date to an int. Seems to work better than previous attemopts of using categorical variables.

ALL = [train, validation, test]
for frame in ALL:
    frame.drop('QuoteNumber', axis=1, inplace=True)
    frame['Original_Quote_Date']= pd.to_datetime(frame['Original_Quote_Date'])
    frame['Original_Quote_Date'] = frame['Original_Quote_Date'].astype(int)
    for c in frame:
        frame[c].fillna(0, inplace=True)
        frame[c].replace(-1, 0, inplace=True)

/Users/blerner/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/blerner/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/blerner/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/blerne

In [43]:
len(validation)

173836

In [40]:
#dropping useless columns
for c in train.drop('QuoteConversion_Flag', axis=1):
    x=train[c].unique()
    if len(x) < 2:
        for frame in ALL:
            frame.drop(c, axis=1, inplace=True)
#rescaling
for c in train.drop('QuoteConversion_Flag', axis=1):
    if train[c].dtype != 'object':
        mean=train[c].mean()
        std = train[c].std()
        if std > 0.0001:
            for frame in ALL:
                frame = (frame[c]-mean)/std

/Users/blerner/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [42]:
#Replacing all categorical variables with dummy variables
for column in train:
    if train[column].dtype == 'object':
        [train, validation, test] = ReplaceWithDummies(column, train, validation, test)
    

/Users/blerner/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/blerner/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
#Run this for partial

X_train=train.drop('QuoteConversion_Flag', axis=1)
X_test=test.drop('QuoteConversion_Flag', axis=1)
X_validation=test.drop('QuoteConversion_Flag', axis=1)
y_train=train['QuoteConversion_Flag']
y_test=test['QuoteConversion_Flag']
y_validation=validation['QuoteConversion_Flag']

In [44]:
#Run this for full
X_train=train.drop('QuoteConversion_Flag', axis=1)
y_train=train['QuoteConversion_Flag']
X_validation= validation

In [49]:
len(X_validation)

173836

In [ ]:
#Support Vector machine
model_svc = svm.SVC()
model_svc.fit(X_train, y_train)
print(sum(y_train))
print(1-sum(abs(np.array(y_train)-np.array(model_svc.predict(X_train))))/float(len(y_train)))

In [48]:
X_train.shape


(260753, 601)

In [19]:
#Poly
pca = PCA(n_components=550) #Instantiate the model & set parameters
pca.fit(X_train); #Fit the model
X_train_red = pca.transform(X_train)
X_validation_red = pca.transform(X_validation)

In [11]:
#introduce interaction terms
poly = PolynomialFeatures()
poly.fit(X_train_red)
X_train_poly=poly.transform(X_train_red)
X_validation_poly = poly.transform(X_validation_red)

In [12]:
X_train_poly.shape

(260753, 1326)

In [ ]:
X_train_red.shape

In [ ]:
#Logistioc regression
model = linear_model.LogisticRegression(C=0.1)
model.fit(X_train, y_train)
predictions=model.predict_proba(X_validation)[:,1]
#print(sum(y_train))
#print(1-sum(abs(np.array(y_train)-np.array(model.predict(X_train))))/float(len(y_train)))
#p1=model.predict(validation)
#print(sum(y_validation))
#print(1-sum(abs(np.array(y_validation)-np.array(model.predict(X_validation))))/float(len(y_validation)))


In [50]:
#Gradient boosting 
model_xgb = xgb.DMatrix(np.array(X_train), label=np.array(y_train))
bst = xgb.train({'objective':'reg:logistic'},dtrain=model_xgb)
predictions = bst.predict(xgb.DMatrix(X_validation))
#predictions = bst.predict(xgb.DMatrix(X_train))
#predictions_binary = []
#for x in list(predictions):
#    if x>=0.5:
#        predictions_binary.append(1)
#    else:
#        predictions_binary.append(0)

#print(sum(y_train))
#print(1-sum(abs(np.array(y_train)-np.array(predictions_binary)))/float(len(y_train)))

In [51]:
len(predictions)

173836

In [ ]:
predictions = bst.predict(xgb.DMatrix(X_validation))
#predictions_binary = []
#for x in list(predictions):
#    if x>=0.5:
#        predictions_binary.append(1)
#    else:
#        predictions_binary.append(0)
#print(sum(y_train))
#print(1-sum(abs(np.array(y_validation)-np.array(predictions_binary)))/float(len(y_validation)))
#p2=predictions_binary

In [ ]:
len(p1)

In [ ]:
#Let's try random forest

rfc_model = RandomForestClassifier(n_estimators = 10, n_jobs=-1)
rfc_model.fit(X_train,y_train)
print(sum(y_train))
print(1-sum(abs(np.array(y_train)-np.array(rfc_model.predict(X_train))))/float(len(y_train)))
#predict2=rfc_model.predict(validation)
#print(sum(y_validation))
#print(1-sum(abs(np.array(y_validation)-np.array(rfc_model.predict(X_validation))))/float(len(y_validation)))
p3=rfc_model.predict(X_validation)

In [ ]:
#Not run for full
ensemble_train =list(np.logical_or(np.array(p1),np.array(p2), np.array(p3)))
ensemble_train = int(ensemble_train==1)
print(1-sum(abs(np.array(y_train)-np.array(ensemble_train)))/float(len(y_train)))

ensemble_validation =list(np.logical_or(np.array(model.predict(X_validation)),np.array(rfc_model.predict(X_validation))))
ensemble_validation = int(ensemble_validation==1)
print(1-sum(abs(np.array(y_validation)-np.array(ensemble_validation)))/float(len(y_validation)))

In [ ]:
ensemble_predict =np.logical_or(np.array(p1),np.array(p2), np.array(p3)).astype(int)
len(ensemble_predict)


In [53]:
#creates the ouput to be submitted
output =pd.DataFrame()
output["QuoteNumber"] = te["QuoteNumber"]
output["QuoteConversion_Flag"] = predictions
output.to_csv("output_boost_newdate.csv", index=False)




In [ ]:
#attempt at neural network: not working right now

model = Sequential()

# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(64, input_dim=584, init='uniform'))
#model.add(Activation('tanh'))
#model.add(Dropout(0.5))
#model.add(Dense(64, init='uniform'))
#model.add(Activation('tanh'))
#model.add(Dropout(0.5))
#model.add(Dense(2, init='uniform'))
#model.add(Activation('softmax'))

#sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer='sgd')

model.fit(X_train1, y_train1)

https://www.kaggle.com/mpearmain/homesite-quote-conversion/xgboost-benchmark/discussion